In [42]:
import pandas as pd
import numpy as np
import re
from ast import literal_eval
from tqdm import tqdm
from ftfy import fix_text
# from util import UnitConversion, mapping_list_values, perl_to_posix
from groupby_toolz import enrich_db, gcloud
from flashtext import KeywordProcessor
from groupby_toolz import enrich_db, gcloud
pd.set_option('display.max_colwidth', None)
import ast

In [2]:
# parameters
customer_id = '5'
formatted_attribute = 'size_diameter_rugs_runners'
attribute = formatted_attribute.lower().replace(' ','_').replace('-','_')

buckets = """Rugs & Mats B"""

# regex_pattern = r"(?i)([^\.]\d* in\w* Belt Width)|(\d*\.\d* in\w* Belt Width)|(\d*\-\d*\/\d* in\w* Belt Width)|(\d*\.\d in\w* Belt Width)"
regex_pattern = r"(?i)(\d+\'\d+)|(\d+\')"


value='%n/a%'
customer_name='%motionindustriesus%'

params = {'customer_id': customer_id ,
          'attribute': attribute,
          'buckets': str(buckets.split('\t'))[1:-1],
          'value':value,
          'customer_name':customer_name
         }
curation_col = f'Q:{attribute}'

In [3]:
def query_from_file(file_name, params):
    with open(f'{file_name}', mode='r') as f:
        text = f.read()
        query = text.format(**params)
        return enrich_db(query)

In [4]:
df = query_from_file(file_name='query/bbby.sql', params=params)
df

attribute value    customers_name external_id  \
0      size_diameter_rugs_runners   n/a  bedbathandbeyond    10212839   
1      size_diameter_rugs_runners   n/a  bedbathandbeyond    10265185   
2      size_diameter_rugs_runners   n/a  bedbathandbeyond    10265207   
3      size_diameter_rugs_runners   n/a  bedbathandbeyond    10265223   
4      size_diameter_rugs_runners   n/a  bedbathandbeyond    10265258   
...                           ...   ...               ...         ...   
96307  size_diameter_rugs_runners   n/a  bedbathandbeyond    77589464   
96308  size_diameter_rugs_runners   n/a  bedbathandbeyond    77589657   
96309  size_diameter_rugs_runners   n/a  bedbathandbeyond    77589658   
96310  size_diameter_rugs_runners   n/a  bedbathandbeyond    77589659   
96311  size_diameter_rugs_runners   n/a  bedbathandbeyond    77589662   

                                                                long_desc  \
0                                                    Multy Home Boot Tray   
1      SALT™ Crystal Clear  52-Inch x 70-Inch Oblong Tablecloth Protector   
2       SALT™ Crystal Clear 70-Inch x 90-Inch Oblong Tablecloth Protector   
3      SALT™ Crystal Clear 70-Inch x 108-Inch Oblong Tablecloth Protector   
4      SALT™ Crystal Clear 70-Inch x 120-Inch Oblong Tablecloth Protector   
...                                                                   ...   
96307     Jaipur Living Thayne Handmade 8'6 x 11'6 Area Rug in Grey/Ivory   
96308         Jaipur Living Bengal 5' x 8' Knotted Area Rug in Grey/Ivory   
96309         Jaipur Living Bengal 4' x 6' Knotted Area Rug in Grey/Ivory   
96310        Jaipur Living Bengal 8' x 10' Knotted Area Rug in Grey/Ivory   
96311        Jaipur Living Bengal 8' x 10' Knotted Area Rug in Blue/Ivory   

      resolution  
0          rules  
1          rules  
2          rules  
3          rules  
4          rules  
...          ...  
96307      rules  
96308      rules  
96309      rules  
96310      rules  
96311      rules  

[96312 rows x 6 columns]

In [5]:
# Find a list of values to iterate
df_strategy = query_from_file(file_name='./asset/easy.sql', params=params) #create query
df_strategy
list_values=df_strategy['value'].to_list()
list_values

['0" - 11.9"',
 '108" or Greater',
 '12" - 23.9"',
 '24" - 35.9"',
 '36" - 47.9"',
 '48" - 59.9"',
 '60" - 71.9"',
 '72" - 83.9"',
 '84" - 95.9"',
 '96" - 107.9"',
 'n/a']

In [6]:
kwp=KeywordProcessor()
kwp.add_keywords_from_list(list_values)
# Extract Key Words
df['long_desc'].apply(lambda x: kwp.extract_keywords(x)) #

0        []
1        []
2        []
3        []
4        []
         ..
96307    []
96308    []
96309    []
96310    []
96311    []
Name: long_desc, Length: 96312, dtype: object

In [7]:
def re_extract(pattern, txt):
    matches = re.findall(pattern, txt)
    tmp_matches = []
    for match in matches:
        for tup in match:
            if tup != '':
                tmp_matches.append(tup)
    return list(set(tmp_matches))

In [8]:
df['matches'] = df['long_desc'].apply(lambda x: re_extract(regex_pattern, x))

In [9]:
df

attribute value    customers_name external_id  \
0      size_diameter_rugs_runners   n/a  bedbathandbeyond    10212839   
1      size_diameter_rugs_runners   n/a  bedbathandbeyond    10265185   
2      size_diameter_rugs_runners   n/a  bedbathandbeyond    10265207   
3      size_diameter_rugs_runners   n/a  bedbathandbeyond    10265223   
4      size_diameter_rugs_runners   n/a  bedbathandbeyond    10265258   
...                           ...   ...               ...         ...   
96307  size_diameter_rugs_runners   n/a  bedbathandbeyond    77589464   
96308  size_diameter_rugs_runners   n/a  bedbathandbeyond    77589657   
96309  size_diameter_rugs_runners   n/a  bedbathandbeyond    77589658   
96310  size_diameter_rugs_runners   n/a  bedbathandbeyond    77589659   
96311  size_diameter_rugs_runners   n/a  bedbathandbeyond    77589662   

                                                                long_desc  \
0                                                    Multy Home Boot Tray   
1      SALT™ Crystal Clear  52-Inch x 70-Inch Oblong Tablecloth Protector   
2       SALT™ Crystal Clear 70-Inch x 90-Inch Oblong Tablecloth Protector   
3      SALT™ Crystal Clear 70-Inch x 108-Inch Oblong Tablecloth Protector   
4      SALT™ Crystal Clear 70-Inch x 120-Inch Oblong Tablecloth Protector   
...                                                                   ...   
96307     Jaipur Living Thayne Handmade 8'6 x 11'6 Area Rug in Grey/Ivory   
96308         Jaipur Living Bengal 5' x 8' Knotted Area Rug in Grey/Ivory   
96309         Jaipur Living Bengal 4' x 6' Knotted Area Rug in Grey/Ivory   
96310        Jaipur Living Bengal 8' x 10' Knotted Area Rug in Grey/Ivory   
96311        Jaipur Living Bengal 8' x 10' Knotted Area Rug in Blue/Ivory   

      resolution      matches  
0          rules           []  
1          rules           []  
2          rules           []  
3          rules           []  
4          rules           []  
...          ...          ...  
96307      rules  [8'6, 11'6]  
96308      rules     [5', 8']  
96309      rules     [4', 6']  
96310      rules    [8', 10']  
96311      rules    [8', 10']  

[96312 rows x 7 columns]

In [46]:
y=pd.DataFrame(df["matches"].to_list(), columns=['col1', 'col2', 'col3', 'col4'])
y=y[['col1', 'col2']]
z=y['col1']

In [50]:
# df['DataFrame Column'] = df['DataFrame Column'].replace(np.nan, 0)
# ['None' if v is None else v for v in z]
y

col1  col2
0      None  None
1      None  None
2      None  None
3      None  None
4      None  None
...     ...   ...
96307   8'6  11'6
96308    5'    8'
96309    4'    6'
96310    8'   10'
96311    8'   10'

[96312 rows x 2 columns]

In [39]:
def parse_ht(ht):
    # format: 7' 0.0"
    ft_,in_= ht.split("' ")
    ht_=ft_*12+in_
#     ft_ = float(ht_[0])
#     in_ = float(ht_[1].replace("\"",""))

    return ht_

In [49]:
y['col1'].apply(lambda x:parse_ht(x))

AttributeError: 'NoneType' object has no attribute 'split'

In [10]:
x['cleaned']=df['matches'].apply(lambda measure: re.sub(r'[a-z]|[A-z]|\s|\w*\.\w*\.|\-','',str(measure)))
max(pd.to_numeric(x['matches'][0:4], errors='coerce'))

NameError: name 'x' is not defined

In [ ]:
matches['matches'] = matches['matches'].apply(lambda measure: re.sub(r'[a-z]|[A-z]|\s|\w*\.\w*\.','',str(measure)))

In [ ]:
error

In [ ]:
df['max']=pd.to_numeric(df['matches'], errors='coerce')
df

In [ ]:
# df['matches']=df[df['matches']!='[]']

In [ ]:
# df['Max']=pd.DataFrame(df['matches'].tolist(),index=df.index).max(1)
y=df[1:4]
# z=y['matches']
maximum=max(y['matches'])
maximum=pd.to_numeric(maximum)
print(max(maximum))

In [ ]:
x=df
x=x[x['matches']!='[]']
x['Max']=pd.DataFrame(df['matches'].tolist(),index=df.index).max(1) 
x

In [ ]:
x['matches'].idxmax()

In [ ]:
df['matches'] = pd.to_numeric(df['matches'])

In [ ]:
df.loc[df['matches'].idxmax()]

In [ ]:
# print(df[df['matches']==df['matches'].max()])

In [ ]:
error

In [ ]:
def na_to_gsheet(df):
    url = 'https://docs.google.com/spreadsheets/d/1A3dXssoEuvgun6YaO5jxM8P6_zZQ9KByEIZlhovF2gU/edit#gid=0'
    wks = 'n/a'
    non_matches = df[df['matches'].astype(str) == '[]']
    non_matches[curation_col] = r'n/a'
    gc = gcloud.Gsheets()
    gc.save_to_gsheet(url, wks, non_matches)

In [ ]:
na_to_gsheet(df)

In [ ]:
matches = df[df['matches'].astype(str) != '[]'] 
x=matches[matches['value']=='n/a']
x

In [ ]:
def remove_txt(lst):
    assert type(lst) == list, "make sure it's a list"
    tmp = []
    for measure in lst:
        val = re.sub(r'[a-z]|[A-z]|\s|\w*\.\w*\.','',measure)
        if val != '':
            tmp.append(val.replace(",",''))
    if str(tmp) == '[]':
        return ''
    else:
        return '{:,}'.format(int(max(tmp)))

In [ ]:
matches['matches'] = matches['matches'].apply(lambda measure: re.sub(r'[a-z]|[A-z]|\s|\w*\.\w*\.','',str(measure)))

In [ ]:
matches['matches'].unique()

In [ ]:
def remove_fractions(numbers):
    if numbers.find('-') > -1:
        numbers = literal_eval(numbers)
        inte, fract = numbers.split('-')
        nom, den = fract.split('/')
        tmp = float(nom)/float(den)
        num = int(inte) + tmp
        return str(num)
    else:
        return numbers

In [ ]:
def remove_fraction(numbers):
    if numbers.find('/') > -1:
        numbers = literal_eval(numbers)
#         inte, fract = numbers.split('-')
        nom, den = numbers.split('/')
        tmp = float(nom)/float(den)
#         num = int(inte) + tmp
        return str(tmp)
    else:
        return numbers

In [ ]:
matches['matches'] = matches['matches'].apply(lambda x: remove_fractions(x))

In [ ]:
matches['matches'].unique()

In [ ]:
matches['matches'] = matches['matches'].apply(lambda x: remove_fraction(x))

In [ ]:
matches['matches'].unique()

In [ ]:
matches['matches'].value_counts()

In [ ]:
matches

In [ ]:
matches['matches'] = matches['matches'].apply(lambda x: float(literal_eval(x)))
matches

In [ ]:
matches[curation_col]=matches['matches']
matches['external_id']=matches['external_id'].apply('="{}"'.format)
matches

In [ ]:
error

In [ ]:
gc = gcloud.Gsheets()

In [ ]:
gc.save_to_gsheet('https://docs.google.com/spreadsheets/d/1A3dXssoEuvgun6YaO5jxM8P6_zZQ9KByEIZlhovF2gU/edit#gid=0','ul',matches)

# send to the folder for upload

In [ ]:
def looks_good(customer, attribute, df, matches):
    drive_path = f'G:/Shared drives/GroupBy Public/Customer Success/.Enrich/Platform Upload Trail/{customer}/_Ready For Upload'
    non_matches = df[df['matches'].astype(str) == '[]']
    non_matches[curation_col] = r'n/a'
    non_matches.to_csv(f'{drive_path}/Motion - {attribute} - na upload_Pulley.csv',index=False)
    matches.to_csv(f'{drive_path}/Motion - {attribute} - match upload_Shawn.csv',index=False)

In [ ]:
looks_good('Motion Industries', attribute, df, matches)